# Reappraisal Training on PyTorch Lightning

## Setup
- When running on Google Colab, mount Google Drive to access scripts.
- `cd` into the project root and install dependencies:

In [ ]:
%load_ext autoreload
%load_ext tensorboard
# from google.colab import drive
# drive.mount('/content/drive')
# %cd {root_dir}

# %pip install pytorch-lightning "ray[tune]" wandb transformers datasets nltk nbdev 
# ! nbdev_install_git_hooks

import nltk
nltk.download('punkt')

ROOT_DIR = '/root/reappraisal-model'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import torch

# Define constants
STRAT = 'far'
BATCH_SIZE = 64 
TEST_BATCH_SIZE = 64
NUM_FOLDS = 5

## Load LDH Data

Contains the following:

- LDHI
- LDHII
- LDHIII

In [ ]:
%autoreload
from reappraisalmodel.ldhdata import LDHDataModule
ldhdata = LDHDataModule(data_dir=ROOT_DIR, batch_size=BATCH_SIZE, strat=STRAT, kfolds=NUM_FOLDS)

Training data loaded from disk.
Encoding Train Data:



Evaluation data loaded from disk.
Encoding Test Data:


## Run K-Fold Training


In [ ]:
%autoreload 2
from reappraisalmodel.trainers import kfold_train


#TODO: ADD CSVLOGGER!!!!
results = kfold_train(NUM_FOLDS, ldhdata, strat=STRAT, 
                       max_epochs=15, 
#                        limit_train_batches=2,
#                        limit_val_batches=1
                    )


Created temporary directory: /tmp/tmpitc3olye


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training on split 1


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory lightning_logs/reappmodel_far_20210227_194439/0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory /tmp/tmpitc3olye/reappmodel_far_20210227_194439 exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory lightning_logs/reappmodel_far_20210227_194439/3 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training on split 5


/opt/conda/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory lightning_logs/reappmodel_far_20210227_194439/4 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)



Successful 0_epoch=04-val_loss=2.09.ckpt to s3: True
{'val_loss': 2.2050764560699463, 'train_loss': 1.891348958015442, 'num_epochs': 7, 'r2score': tensor(0.1142), 'explained_var': tensor(0.3996)}
Successful 1_epoch=06-val_loss=2.01.ckpt to s3: True
{'val_loss': 2.1231682300567627, 'train_loss': 1.8152025938034058, 'num_epochs': 9, 'r2score': tensor(0.1057), 'explained_var': tensor(0.3687)}
Successful 2_epoch=01-val_loss=1.94.ckpt to s3: True
{'val_loss': 3.1561508178710938, 'train_loss': 2.0191667079925537, 'num_epochs': 4, 'r2score': tensor(-0.2645), 'explained_var': tensor(0.3380)}
Successful 3_epoch=05-val_loss=1.96.ckpt to s3: True
{'val_loss': 2.0250930786132812, 'train_loss': 1.8771026134490967, 'num_epochs': 8, 'r2score': tensor(0.1549), 'explained_var': tensor(0.4218)}
Successful 4_epoch=08-val_loss=1.76.ckpt to s3: True
{'val_loss': 1.882969617843628, 'train_loss': 1.768418788909912, 'num_epochs': 11, 'r2score': tensor(0.2085), 'explained_var': tensor(0.4261)}


In [ ]:
import pandas as pd

pd.DataFrame(results)

## Tuning Hyperparameters


In [ ]:
%autoreload
from reappraisalmodel.trainers import run_tune
run_tune(ldhdata)

# Exploring

## Shape of Model

In [ ]:
pretrained_model = "distilbert-base-uncased-finetuned-sst-2-english"

from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained(pretrained_model)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

### Tokenization

In [ ]:
# hide
# Returns a BatchEncoding of the text.
tokenized = tokenizer(text = ["This is the first test sentence!", "This is the second, better test sentence."], 
    padding='max_length', max_length=150)

for idx, sent in enumerate(tokenized.input_ids):
    print(f"Sentence            {idx}: {tokenizer.convert_ids_to_tokens(sent)}")
    print(f"Tokenized Attention {idx}: {tokenized[idx].attention_mask}")



### Training

In [ ]:
%autoreload
import torch
import pytorch_lightning as lit
from reappraisalmodel.lightningreapp import LightningReapp

default_config = default_config = {
    'lr': 1e-3,
    'hidden_layer_size': 50
}

model = LightningReapp(default_config)

trainer = lit.Trainer(
    gpus = 1 if torch.cuda.is_available() else None,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    max_epochs=10,
    fast_dev_run=2,
    terminate_on_nan=True)

model = LightningReapp(default_config)

trainer.fit(model, ldhdata.train_dataloader(), ldhdata.val_dataloader())



In [ ]:
model = LightningReapp.load_from_checkpoint(
    '/Users/danielpham/Google Drive/ldh/lightning_logs_obj_0223/version_2/checkpoints/epoch=1-step=337.ckpt', map_location='cpu')

model.eval()
model

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer 

model.eval()
trainer = Trainer(
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    terminate_on_nan=True)

test_dataloader = ldhdata

In [ ]:
dfs = []
for result in results:
    print(len(result['predict']))

In [ ]:
import pickle
with open('./output_reapp.pkl', 'rb+') as f:
    results = pickle.load(f)

In [ ]:
import boto3
import pickle

from sagemaker import get_execution_role
role = get_execution_role()

bucket = 'ldhdata'
file = 'Master_Final_TrainingData.csv'

s3client = boto3.client('s3')

response = s3client.get_object(Bucket=bucket, Key=file)

import codecs 
import csv

train = csv.DictReader(codecs.getreader("utf-8")(response["Body"])) # returns an ordered dict


In [ ]:
import torch
from pytorch_lightning.metrics.functional import r2score, explained_variance

expected = torch.rand(16)
observed = torch.rand(16)

r2score(expected, observed)

In [ ]:
!nbdev_update_lib